In [1]:
  #!/usr/bin/env python
#Numcosmo import
try:
    import gi
    gi.require_version('NumCosmo', '1.0')
    gi.require_version('NumCosmoMath', '1.0')
except:
    pass

import math
import matplotlib.pyplot as plt
from gi.repository import GObject
from gi.repository import NumCosmo as Nc
from gi.repository import NumCosmoMath as Ncm
import sys

# General imports
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from astropy.table import Table
from astropy.io import fits
import pandas as pd

# DC2 catalog-related imports
import FoFCatalogMatching
import GCRCatalogs
from GCR import GCRQuery

Ncm.cfg_init()
Ncm.cfg_set_log_handler (lambda msg: sys.stdout.write (msg) and sys.stdout.flush ())

In [2]:
#A list with all DC2 catalogs available

GCRCatalogs.get_available_catalog_names(include_default_only=False)

['base5000_v1.1.1',
 'base5000_v1.1.1_9431_9812',
 'base5000_v1.1.1_9556',
 'base5000_v1.1.1_image',
 'base5000_v1.1.1_small',
 'base5000_v1.1.1_z_0_1',
 'base5000_v1.1.1_z_1_2',
 'baseDC2_snapshot_z0.15_v0.1',
 'baseDC2_snapshot_z0.15_v0.1_small',
 'baseDC2_snapshot_z1.01_v0.1',
 'baseDC2_v1.1_9431_9812',
 'baseDC2_v1.1_9556',
 'baseDC2_v1.1_image',
 'buzzard',
 'buzzard_high-res',
 'buzzard_high-res_v1.1',
 'buzzard_test',
 'buzzard_v2.0.0_3',
 'buzzard_v2.0.0_4',
 'buzzard_v2.0.0_test',
 'cosmoDC2',
 'cosmoDC2_v1.0',
 'cosmoDC2_v1.0_10194_10452',
 'cosmoDC2_v1.0_8786_9049',
 'cosmoDC2_v1.0_9050_9430',
 'cosmoDC2_v1.0_9431_9812',
 'cosmoDC2_v1.0_9556',
 'cosmoDC2_v1.0_9813_10193',
 'cosmoDC2_v1.0_image',
 'cosmoDC2_v1.0_image_addon_knots',
 'cosmoDC2_v1.0_small',
 'cosmoDC2_v1.1.4',
 'cosmoDC2_v1.1.4_image',
 'cosmoDC2_v1.1.4_image_addon_knots',
 'cosmoDC2_v1.1.4_image_combined_agn',
 'cosmoDC2_v1.1.4_image_with_photoz_calib',
 'cosmoDC2_v1.1.4_image_with_photozs_flexzboost_v1',
 'co

In [3]:
#We are loading the cosmoDC2 catalalog with photoz
redmapper_cat = GCRCatalogs.load_catalog('cosmoDC2_v1.1.4_redmapper_v0.8.1')

#Taking some important information about the fiducial cosmological model and survey region
cosmo_dc2 = redmapper_cat.cosmology
sky_area = redmapper_cat.sky_area

#Listing all quantities in the choosen catalog
np.sort(redmapper_cat.list_all_quantities())

array(['cluster_id', 'cluster_id_member', 'dec', 'dec_cen_0', 'dec_cen_1',
       'dec_cen_2', 'dec_cen_3', 'dec_cen_4', 'dec_member', 'id_cen_0',
       'id_cen_1', 'id_cen_2', 'id_cen_3', 'id_cen_4', 'id_member',
       'mag_g_lsst_member', 'mag_i_lsst_member', 'mag_r_lsst_member',
       'mag_y_lsst_member', 'mag_z_lsst_member', 'magerr_g_lsst_member',
       'magerr_i_lsst_member', 'magerr_r_lsst_member',
       'magerr_y_lsst_member', 'magerr_z_lsst_member', 'maskfrac',
       'p_cen_0', 'p_cen_1', 'p_cen_2', 'p_cen_3', 'p_cen_4', 'p_member',
       'pfree_member', 'ra', 'ra_cen_0', 'ra_cen_1', 'ra_cen_2',
       'ra_cen_3', 'ra_cen_4', 'ra_member', 'redshift', 'redshift_err',
       'redshift_true_cg', 'redshift_true_member', 'refmag_err_member',
       'refmag_member', 'richness', 'richness_err', 'scaleval',
       'theta_i_member', 'theta_r_member'], dtype='<U20')

In [4]:
z_min = 0
z_max = 1
richness_min = 0
richness_max = 5.5
data = redmapper_cat.get_quantities(['redshift','redshift_err','richness','richness_err','ra','dec'])


redshift_err = data['redshift_err']
richness = data['richness']
richness_err = data['richness_err']
redshift = data['redshift']
ra = data['ra']
dec = data['dec']

ncountdata = {'richness': np.log(richness), 'richness_err': np.log(richness_err),'z': redshift,'z_err': redshift_err,'ra':ra,'dec':dec}
ncountdata = pd.DataFrame(ncountdata)

richness_mask_min = (richness_min <= ncountdata['richness'])
richness_mask_max = (richness_max >= ncountdata['richness'] )

z_mask_min = (z_min <=ncountdata['z'])
z_mask_max = (z_max >=ncountdata['z'])

mask =  z_mask_min & z_mask_max & richness_mask_min & richness_mask_max

ncountdata = ncountdata[mask]




N_cl = len(ncountdata)
print(f'There are {N_cl} halos with Richness= [{np.exp(richness_min):.2e},{np.exp(richness_max):.2e}] and z=[{z_min},{z_max}] in {redmapper_cat.sky_area} deg2')
display(ncountdata)




There are 38705 halos with Richness= [1.00e+00,2.45e+02] and z=[0,1] in 439.78987 deg2


,richness,richness_err,z,z_err,ra,dec
0,5.250736,1.320947,0.513513,0.002786,52.520436,-25.717996
1,5.188101,1.306557,0.571270,0.003218,56.920377,-27.204341
3,5.120004,1.302292,0.635185,0.006033,58.563624,-27.429962
4,4.717737,1.213890,0.442303,0.003674,58.355198,-26.867687
5,4.648611,1.084195,0.466972,0.003501,57.765757,-28.119102
...,...,...,...,...,...,...
45070,1.717804,0.408361,0.594197,0.014232,68.391998,-45.515891
45074,1.781440,0.233527,0.471931,0.006561,69.518360,-44.856106
45075,1.664387,0.210837,0.501281,0.008677,70.472970,-44.819294
45078,1.611006,-0.107732,0.211521,0.006237,71.592885,-44.833270


# Numcosmo Cosmology

In [5]:
cosmo = Nc.HICosmo.new_from_name(Nc.HICosmo, "NcHICosmoDEXcdm")
reion = Nc.HIReionCamb.new () 

prim = Nc.HIPrimPowerLaw.new () 

cosmo.add_submodel (reion)
cosmo.add_submodel (prim)

dist = Nc.Distance.new(2.0)

tf = Nc.TransferFunc.new_from_name('NcTransferFuncEH')

psml = Nc.PowspecMLTransfer.new (tf)
psml.require_kmin (1.0e-3)
psml.require_kmax (1.0e3)
psf = Ncm.PowspecFilter.new (psml, Ncm.PowspecFilterType.TOPHAT)
psf.set_best_lnr0 ()

mulf = Nc.MultiplicityFuncBocquet.new ()
mulf.set_mdef (Nc.MultiplicityFuncMassDef.CRITICAL)
mulf.set_Delta (200.0)
mulf.set_sim (Nc.MultiplicityFuncBocquetSim.DM)

hmf = Nc.HaloMassFunction.new (dist, psf, mulf)
hmf.set_area(redmapper_cat.sky_area)

In [6]:
#Numcosmo_proxy model
cluster_z = Nc.ClusterRedshift.new_from_name("NcClusterRedshiftNodist{'z-min': <%20.15e>, 'z-max':<%20.15e>}" % (z_min, z_max))
cluster_m = Nc.ClusterMass.new_from_name("NcClusterMassAscaso{'M0':<%20.15e>,'z0':<%20.15e>,'lnRichness-min':<%20.15e>, 'lnRichness-max':<%20.15e>}" % (3e14/(0.71),0.6, richness_min, richness_max))
cluster_m.param_set_by_name('mup0', 3.19)
cluster_m.param_set_by_name('mup1', 2/np.log(10))
cluster_m.param_set_by_name('mup2', -0.7/np.log(10))
cluster_m.param_set_by_name('sigmap0', 0.33)
cluster_m.param_set_by_name('sigmap1', -0.08/np.log(10))
cluster_m.param_set_by_name('sigmap2', 0/np.log(10))

In [7]:
ca = Nc.ClusterAbundance.new(hmf,None)
ncdata = Nc.DataClusterNCount.new (ca, "NcClusterRedshiftNodist", "NcClusterMassAscaso")
ca.prepare(cosmo,cluster_z,cluster_m)
mset = Ncm.MSet.new_array ([cosmo, cluster_z, cluster_m])

richness = np.array(ncountdata['richness'])
z = np.array(ncountdata['z'])
ncdata.set_lnM_obs(Ncm.Matrix.new_array (richness,1))
ncdata.set_z_obs(Ncm.Matrix.new_array (z,1))

ncdata.true_data (False)
ncdata.set_init(True)

#Bins definition
z_edges = np.linspace(z_min, z_max, 5)
proxy_edges = np.linspace(richness_min, richness_max, 4)

ncdata.del_bins ()
for rich_l, rich_u in zip(proxy_edges, proxy_edges[1:]):
    for zl, zu in zip(z_edges,z_edges[1:]):
        lnM_obs_lb = Ncm.Vector.new_array ([rich_l])
        lnM_obs_ub = Ncm.Vector.new_array ([rich_u])
        z_obs_lb = Ncm.Vector.new_array ([zl])
        z_obs_ub = Ncm.Vector.new_array ([zu])
        
        ncdata.add_bin (lnM_obs_lb, lnM_obs_ub, z_obs_lb, z_obs_ub)

ncdata.bin_data ()

# #
# # Save to a fits file
# #
ncdata.catalog_save ("redmapper_richness_ncounts.fits", True)

In [8]:
ncdata_fits = fits.open('redmapper_richness_ncounts.fits')
ncdata_fits.info()

ncdata_unbinned_data =  ncdata_fits[1].data
ncdata_binned_data = ncdata_fits[2].data

ncdata_unbinned_Table = Table(ncdata_unbinned_data)
display(ncdata_unbinned_Table)

ncdata_binned_Table = Table(ncdata_binned_data)
display(ncdata_binned_Table)

Filename: redmapper_richness_ncounts.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       6   ()      
  1  ClusterAbundance    1 BinTableHDU     19   38705R x 2C   [1D, 1D]   
  2  ClusterAbundance    1 BinTableHDU     25   12R x 5C   [1D, 1D, 1D, 1D, 1D]   


Z_OBS,LNM_OBS
float64,float64
0.513512909412384,5.250736236572266
0.571269690990448,5.188101291656494
0.6351852416992188,5.120004177093506
0.44230303168296814,4.717737197875977
0.4669721722602844,4.648610591888428
0.4413633942604065,4.63951301574707
0.5675368309020996,4.495318412780762
0.500461220741272,4.349089622497559
0.49930539727211,4.410985469818115


Z_OBS_LB,Z_OBS_UB,LNM_OBS_LB,LNM_OBS_UB,BIN_COUNT
float64,float64,float64,float64,float64
0.0,0.25,0.0,1.8333333333333333,578.0
0.25,0.5,0.0,1.8333333333333333,3377.0
0.5,0.75,0.0,1.8333333333333333,4616.0
0.75,1.0,0.0,1.8333333333333333,3675.0
0.0,0.25,1.8333333333333333,3.6666666666666665,1059.0
0.25,0.5,1.8333333333333333,3.6666666666666665,6579.0
0.5,0.75,1.8333333333333333,3.6666666666666665,9510.0
0.75,1.0,1.8333333333333333,3.6666666666666665,8304.0
0.0,0.25,3.6666666666666665,5.5,55.0
